In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time

In [2]:
fileName ="Vehicle Check/Northern Branch Phase II Debris Removal Ops.xlsx"
df = pd.read_excel(fileName)

In [ ]:
df.to_excel("Test.xlsx")

# See  Columns we need for our data analysis

In [3]:
df.columns.tolist()

['APN',
 'Structural Status',
 'Haz Tree Only',
 '# of Trees',
 'Haz Trees Assessment',
 'Haz Trees',
 'Hazard Tree Status',
 'Street #',
 'Unit #',
 'Street Name',
 'City',
 'County',
 'Division',
 'Hold Reason',
 'Hold',
 'Gate Locked?',
 'Code/Key Location',
 'ROE Verified',
 'Declaration',
 'Property Type',
 'Commercial Notes',
 'Scheduled',
 'Consultant',
 'Debris Crew',
 'Debris Crew Leader/Crew#',
 'Debris Crew WO#',
 'ROE Date',
 'Priority',
 'Withdrawal',
 'Not Eligible',
 'Reason for Ineligibility',
 'Determinant for Ineligibility',
 'Safety Concern Property',
 'Phase 1 Complete',
 'Needs Milk Run or Phase 1 Revisit',
 'Milk Run/Phase 1 Revisit Complete',
 'Access Issues',
 'EHP Attention Required',
 'EHP Attn Notes',
 'Monitoring Needed',
 'Monitoring Needed Description',
 'Bio Assessment Notes',
 'Arch Assessment Notes',
 'CalOES Review Notes',
 'Permit Needed',
 'Permit Description',
 'Owner Name',
 'Contact Phone',
 'Contact Email',
 'Site Assessment',
 'RAD/MERC Sweep',


# We have text in a number column(FILLNA), and we will need to covert from object to a numeric value for Number of Vehicles Removed & Number of Vehicles. 

In [ ]:
df.loc[:, ['Number of Vehicles', 'Number of Vehicles Removed']] = df.loc[:,['Number of Vehicles', 'Number of Vehicles Removed']].fillna(0)
df.loc[:, "Number of Vehicles"] = pd.to_numeric(df.loc[:, "Number of Vehicles"], errors='coerce')
df.loc[:, "Number of Vehicles Removed"] = pd.to_numeric(df.loc[:,"Number of Vehicles Removed"], errors='coerce')
# df.to_excel("Removed NAN test.xlsx")

# For Butte only
## More filters before geting our numbers
### Fill in NAN with zeros-(DONE)
### Take out all withdramn and Ineligible
### num of vecs - only properties with num of vecs removed = 0

In [ ]:
# df[['Number of Vehicles', 'Number of Vehicles Removed']] = df[['Number of Vehicles', 'Number of Vehicles Removed']].fillna(0)
Status = ("Withdrawal", "Ineligible")

# df['Number of Vehicles Removed'] = df[(df['Number of Vehicles Removed'] == 0)]

df = df[(df.loc[:, 'Number of Vehicles Removed'] == 0) &  (df.loc[:, "County"] == "Butte") & (~df.loc[:, 'Structural Status'].isin(Status))]
df.loc[:, "Number of Vehicles"] = pd.to_numeric(df.loc[:, "Number of Vehicles"], errors='coerce')
df.loc[:, "Number of Vehicles Removed"] = pd.to_numeric(df.loc[:, "Number of Vehicles Removed"], errors='coerce')



# df.to_excel("Check nums with SS.xlsx")

# Check for our data types before graphing/Doing any arthimatic

In [ ]:
df.dtypes[['Number of Vehicles', 'Number of Vehicles Removed', 'County']]

# This will group data by county and then only show the comlumns we need
## This couts up the number in the column, this case columns, that we watn to show by county

##### In general, use COUNT() when you want to count how many rows contain a non-empty value for a specified column. Use SUM() when you want to get the total sum of all values in a column

In [ ]:
df["Totals"] = df.loc[:, 'Number of Vehicles'] - df.loc[:, 'Number of Vehicles Removed']

df.groupby('County')[['Number of Vehicles', 'Number of Vehicles Removed', "Totals"]].sum()

# df.to_excel("Todays Vehicles by county.xlsx")

# Same as above but for both ways of doing it be sure to fill in the NAN. Not sure way but the .sum() doesnt act right if there are NAN

In [ ]:
sumvecs = df.groupby("County").sum()
filt = sumvecs.loc[:]
filt

# Use the above in one function. (ADD to pycharm)

In [ ]:
def sumVehicles(df):
    df = df.copy()
# We have text in a number column(FILLNA), and we will need to covert from object to a numeric value for Number of Vehicles Removed & Number of Vehicles
    df.loc[:, ['Number of Vehicles', 'Number of Vehicles Removed']] = df.loc[:,['Number of Vehicles', 'Number of Vehicles Removed']].fillna(0)
    df.loc[:, "Number of Vehicles"] = pd.to_numeric(df.loc[:, "Number of Vehicles"], errors='coerce')
    df.loc[:, "Number of Vehicles Removed"] = pd.to_numeric(df.loc[:,"Number of Vehicles Removed"], errors='coerce')
    
#vairable for isin()
    Status = ("Withdrawal", "Ineligible")

# filter for butte county only, number of vhelice remove 0 only, take out "status" aka withdrawals anf ineligible
    df = df[(df.loc[:, 'Number of Vehicles Removed'] == 0) &  (df.loc[:, "County"] == "Butte") & (~df.loc[:, 'Structural Status'].isin(Status))]
    df.loc[:, "Number of Vehicles"] = pd.to_numeric(df.loc[:, "Number of Vehicles"], errors='coerce')
    df.loc[:, "Number of Vehicles Removed"] = pd.to_numeric(df.loc[:, "Number of Vehicles Removed"], errors='coerce')
    
# Get out math stuff done nuber of vehicles minus vechilces removed. See how many is left
    df.loc[:, ["Vehicles Left"]] = df.loc[:, "Number of Vehicles"] - df.loc[:, "Number of Vehicles Removed"]
    
# sums up all our counts for the columns named
    return df.groupby("County")[["Number of Vehicles", "Number of Vehicles Removed", "Vehicles Left"]].sum()
#     df.to_excel("Removed NAN test.xlsx")
sumVehicles(df)

# Mark a double bar graph from the numbers above(WIP)

In [ ]:
# county = df.groupby(["County"])[['Number of Vehicles','Number of Vehicles Removed']].count()

# vehicles = county.loc[:, ['Number of Vehicles','Number of Vehicles Removed']]

df.groupby("County")[['Number of Vehicles','Number of Vehicles Removed']].count().unstack(0).plot.bar()
# for num in range(len(county)):
#     plt.text(num, vehicles[num], vehicles[num], ha='center', va='bottom')
plt.show()

In [ ]:
county = df.groupby(["County"])[['Number of Vehicles','Number of Vehicles Removed']].count()

numVehicles = county.loc[:, 'Number of Vehicles']

numVehicles.plot(kind='bar')
for num in range(len(county)):
    plt.text(num, numVehicles[num], numVehicles[num], ha='center', va='bottom')
plt.show()